<a href="https://colab.research.google.com/github/Joshuajee/AI-ML-PROJECTS/blob/master/GET_DATA_From_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gpt4all
!pip install "gpt4all[cuda]"

In [ ]:
import requests
import json
from google.colab import drive
from gpt4all import GPT4All
import torch

has_gpu = torch.cuda.is_available()

has_gpu  # Should return True

In [ ]:
drive.mount('/content/drive/') # Mount your Google Drive to the local /content/drive directory
drive_path = "/content/drive/My Drive/Colab Notebooks/assessment-1/data/"

In [ ]:
def create_llm_response(model, batch_size = 4):
    '''
    This function go through the prompt CSV file and
    save the response of the LLM inside a CSV file.
    it always start from where it stops so canceling the process will not lead to lose
    '''
    if has_gpu:
        gpt_model = GPT4All(model, device="cuda") # downloads / loads a 4.66GB LLM and run on GPU
    else:
        gpt_model = GPT4All(model) # downloads / loads a 4.66GB LLM

    model_file = open(drive_path + model + ".csv", "a+")
    model_file.seek(0)
    last_line = len(model_file.readlines())
    model_file.seek(last_line)
    current_line = last_line
    with open(drive_path + "/prompt.csv", "r") as file:
        lines = file.readlines()
        while current_line <= 1000:
            print(f"Job progress: {current_line/10} %")
            line_split = lines[current_line].split(",")
            prompt = "Write an article on " + line_split[2].replace("\n", "")
            if current_line == 0:
                model_file.write(line_split[0] + "," + line_split[1] + "," + line_split[2].replace('\n', '') + ", Prompt, Response, Model\n")
            else:
                with gpt_model.chat_session():
                    answer = gpt_model.generate(prompt, max_tokens=1024)
                    answer = answer.replace("\n", "").replace('"', "'")
                    model_file.write(line_split[0] + "," + line_split[1] + ',"' + str((line_split[2]).strip()) +  '","' + str(prompt.strip()) + '","' + str(answer.strip()) + '",' + model + " \n")
            current_line += 1
            #To save after every 10 lines that is processed
            if current_line % batch_size == 0:
                model_file.close()
                model_file = open(drive_path + model + ".csv", "a+")
                print(f"Batch {(current_line // batch_size) + 1} saved")
        print(f"Job completed")
    model_file.close()

In [ ]:
create_llm_response("Meta-Llama-3-8B-Instruct.Q4_0.gguf")

In [ ]:
create_llm_response("Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf")

Downloading: 100%|██████████| 4.11G/4.11G [00:23<00:00, 172MiB/s]
Verifying: 100%|██████████| 4.11G/4.11G [00:19<00:00, 209MiB/s]


Job progress: 42.0 %
Job progress: 42.1 %
Job progress: 42.2 %
Job progress: 42.3 %
Batch 107 saved
Job progress: 42.4 %
Job progress: 42.5 %
Job progress: 42.6 %
Job progress: 42.7 %
Batch 108 saved
Job progress: 42.8 %
Job progress: 42.9 %
Job progress: 43.0 %
Job progress: 43.1 %
Batch 109 saved
Job progress: 43.2 %
Job progress: 43.3 %
Job progress: 43.4 %
Job progress: 43.5 %
Batch 110 saved
Job progress: 43.6 %
Job progress: 43.7 %
Job progress: 43.8 %
Job progress: 43.9 %
Batch 111 saved
Job progress: 44.0 %
Job progress: 44.1 %
Job progress: 44.2 %
Job progress: 44.3 %
Batch 112 saved
Job progress: 44.4 %
Job progress: 44.5 %
Job progress: 44.6 %
Job progress: 44.7 %
Batch 113 saved
Job progress: 44.8 %
Job progress: 44.9 %
Job progress: 45.0 %
Job progress: 45.1 %
Batch 114 saved
Job progress: 45.2 %
Job progress: 45.3 %
Job progress: 45.4 %
Job progress: 45.5 %
Batch 115 saved
Job progress: 45.6 %
Job progress: 45.7 %
Job progress: 45.8 %
Job progress: 45.9 %
Batch 116 saved


In [ ]:
create_llm_response("Phi-3-mini-4k-instruct.Q4_0.gguf")

Downloading: 100%|██████████| 2.18G/2.18G [00:33<00:00, 65.5MiB/s]
Verifying: 100%|██████████| 2.18G/2.18G [00:11<00:00, 191MiB/s]


Job progress: 77.6 %
Job progress: 77.7 %
Job progress: 77.8 %
Job progress: 77.9 %
Batch 196 saved
Job progress: 78.0 %
Job progress: 78.1 %
Job progress: 78.2 %
Job progress: 78.3 %
Batch 197 saved
Job progress: 78.4 %
Job progress: 78.5 %
Job progress: 78.6 %
Job progress: 78.7 %
Batch 198 saved
Job progress: 78.8 %
Job progress: 78.9 %
Job progress: 79.0 %
Job progress: 79.1 %
Batch 199 saved
Job progress: 79.2 %
Job progress: 79.3 %
Job progress: 79.4 %
Job progress: 79.5 %
Batch 200 saved
Job progress: 79.6 %
Job progress: 79.7 %
Job progress: 79.8 %
Job progress: 79.9 %
Batch 201 saved
Job progress: 80.0 %
Job progress: 80.1 %
Job progress: 80.2 %
Job progress: 80.3 %
Batch 202 saved
Job progress: 80.4 %
Job progress: 80.5 %
Job progress: 80.6 %
Job progress: 80.7 %
Batch 203 saved
Job progress: 80.8 %
Job progress: 80.9 %
Job progress: 81.0 %
Job progress: 81.1 %
Batch 204 saved
Job progress: 81.2 %
Job progress: 81.3 %
Job progress: 81.4 %
Job progress: 81.5 %
Batch 205 saved


In [ ]:
create_llm_response("orca-mini-3b-gguf2-q4_0.gguf", 1)

Downloading: 100%|██████████| 1.98G/1.98G [00:49<00:00, 40.0MiB/s]
Verifying: 100%|██████████| 1.98G/1.98G [00:04<00:00, 473MiB/s]


Job progress: 0.0 %
Batch 2 saved
Job progress: 0.1 %
Batch 3 saved
Job progress: 0.2 %
Batch 4 saved
Job progress: 0.3 %
Batch 5 saved
Job progress: 0.4 %
Batch 6 saved
Job progress: 0.5 %
Batch 7 saved
Job progress: 0.6 %
Batch 8 saved
Job progress: 0.7 %
Batch 9 saved
Job progress: 0.8 %
Batch 10 saved
Job progress: 0.9 %
Batch 11 saved
Job progress: 1.0 %
Batch 12 saved
Job progress: 1.1 %
Batch 13 saved
Job progress: 1.2 %
Batch 14 saved
Job progress: 1.3 %
Batch 15 saved
Job progress: 1.4 %
Batch 16 saved
Job progress: 1.5 %
Batch 17 saved
Job progress: 1.6 %
Batch 18 saved
Job progress: 1.7 %
Batch 19 saved
Job progress: 1.8 %
Batch 20 saved
Job progress: 1.9 %
Batch 21 saved
Job progress: 2.0 %
Batch 22 saved
Job progress: 2.1 %
Batch 23 saved
Job progress: 2.2 %
Batch 24 saved
Job progress: 2.3 %
Batch 25 saved
Job progress: 2.4 %
Batch 26 saved
Job progress: 2.5 %
Batch 27 saved
Job progress: 2.6 %
Batch 28 saved
Job progress: 2.7 %
Batch 29 saved
Job progress: 2.8 %
Batch 30

In [ ]:
create_llm_response("gpt4all-13b-snoozy-q4_0.gguf")

In [ ]:
!nvidia-smi